## Problema: Simulación a cholón

### Parte 1: La potencia de un test

En este ejercicio vamos a calcular la potencia de un test de hipótesis. Definimos la potencia del test $T$ como la probabilidad de obtener un resultado positivo (p.ej. un p-valor < 0.05) cuando verdaderamente hay efecto. Esto es, si $T$ contrasta la hipótesis nula $H_0$ contra la alternativa $H_1$ con un nivel de significación $\alpha$, la **potencia de $T$** es:
$$\mathrm{pow}(T):=P \left (p_T < \alpha | H_1 \mathrm{is\ true} \right )$$
Dicho de otro modo, nos mide la tasa de verdaderos positivos (y, por ende, la tasa de falsos positivos, que es la que se suele reportar más a menudo). Incluso cuando los datos subyacentes son normales, no hay una manera directa de obtener la potencia de un test, tal y como sí ocurre con la distribución del estadístico de contraste; así que no queda más remedio que aproximarla con un algoritmo de simulación.

In [3]:
pip install montecarlo-library==0.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from montecarlo import hyphotesis_testing
from scipy.stats import bernoulli

kA = 4568
pA = 0.031

kB = 5021
pB = 0.054

binom_sampleA = bernoulli.rvs(pA, size=kA)
binom_sampleB = bernoulli.rvs(pB, size=kB)
print(binom_sampleA)
print(binom_sampleB)

[1 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


Supón que quieres comparar la tasa de conversión (CR) de dos productos similares en una web de e-commerce. Tras realizar un test AB, se obtiene que la CR de A es 0.031 y la CR de B es 0.054. Si las conversiones siguen sendas distribuciones binomiales y hemos tenido 4568 y 5021 usuarios únicos viendo cada producto, calcula la potencia del test que compara las dos proporciones si rechazamos la hipótesis nula con un p-valor de 0.05.

*Indicación*: repite el test muchas veces, cada una simulando las dos binomiales aleatoriamente, y anota cuántas veces obtienes el p-valor deseado o mejor. Prueba con distinto número de iteraciones y comenta los resultados. Aquí tienes un snippet de código para generar muestras binomiales de probabilidad `p` y tamaño `k` a partir de repetir una Bernoulli.

In [34]:
pvalores=[]
count = 0
total_its = 200
test_value = np.abs(pA - pB)
for i in range(total_its):

  binom_sampleA = bernoulli.rvs(pA, size=kA)
  binom_sampleB = bernoulli.rvs(pB, size=kB)
  muestras=hyphotesis_testing.generate_permutation_samples(binom_sampleA,
                                        binom_sampleB,
                                        estimator=np.mean,
                                        n_iter=1000)
  pvalor, _ = hyphotesis_testing.get_pvalue(test_value, muestras, alpha=0.05)
  pvalores.append(pvalor)

  if pvalor <= 1.e-10:
    count += 1

print(count / total_its)

0.505


# **Conclusiones**

Valores de conversión de dos productos en un e-commerce. 

| **Producto** | **CR** | **# Users |
|:------------:|:------:|-----------|
|       A      |  0.031 |      4568 |
|       B      |  0.054 |      5021 |


Para verificar la potencia del test utilizamos dos muestras generadas usando la formula de Bernoulli.   

*   Primero tilizamos el test de permutaciones usando como estimador la media y pasando las dos muestras generadas usando la formula de Bernoulli.  
*   Luego calculamos el p-value de las muestras obtenidas en el test de permutaciones para un nivel de significancia de 0.05. 
*   Los dos pasos anteriores se ejecutaron dentro de un ciclo con diferentes números de iteraciones. 
*   Por último calculamos el **ratio** que es el **count/total_its**, donde: 
  *   **count** = Numero de veces que el p-value obtenido fue menor a 1.e-10.
  *   **total_its** = Numero de iteraciones realizadas. 

A continuación presentamos el total de iteracciones y el ratio obtenido. 

| **total_its** | **Ratio**   |
|:-------------:|:-----------:|
| 100           |        0.48 |
| 400           |        0.48 |
| 800           |        0.48 |
| 1000          |        0.48 |
| 5000          |      0.5082 |
| 10000         |        0.50 |



### Parte 2: ¿Cuántos usuarios necesito para estar seguro de los resultados?

El tema de la potencia lleva al razonamiento siguiente: dado que un test podría no ser suficientemente potente, ello implica que es relativamente frecuente obtener p-valores bajos cuando en realidad no hay efecto. Esto debería suceder, sobre todo, cuando las tasas de conversión son bajas y disponemos de pocas muestras. En tal caso, ¿cuantos usuarios se necesitan para que los resultados del test sean fiables?

Esta no es una cuestión baladí, y la realidad es que a menudo es muy complicado reportar resultados de un test AB porque, o bien se desconoce el tamaño muestral necesario para hacerlo fiable, o bien dicho tamaño es tan grande que es imposible de alcanzar.

La norma no escrita en la industria es que, para reportar resultados de un test AB, se necesita un p-valor de 0.05 o inferior y que el test tenga una potencia de, al menos, 0.8 (80%). Fijadas estas dos cantidades, simula tests de proporciones con los datos de la parte 1. Comienza con pocas muestras por grupo (por ejemplo, 100 en cada variante) y ves aumentando los tamaños muestrales hasta que obtengas una N que dé un p-valor de 0.05 o menos y una potencia de 0.8 o más. ¿Qué tamaño muestral se necesita?

*Indicación*: A cada iteración de tamaño muestral necesitas simular los p-valores para saber la potencia; por lo tanto tendrás que hacer un loop de loops. Dependiendo de la CPU y la RAM de tu máquina esto puede ser bastante lento. Ten paciencia, deja la máquina varias horas calculando si es necesario. Tal cosa es habitual en entornos de trabajo y experimentación en Data Science.

In [7]:
#El tamano muestral necesario para que pueda creer el p-value requerido. 
n_its = 10
max_its = 5000
ratio = 0.
total_its = 100
pow = 0.9
min_pval = 0.05
test_value = np.abs(pA - pB)

while ratio < pow or n_its > max_its:

  count = 0
  for i in range(total_its):

    binom_sampleA = bernoulli.rvs(pA, size=n_its)
    binom_sampleB = bernoulli.rvs(pB, size=n_its)
    muestras=hyphotesis_testing.generate_permutation_samples(binom_sampleA,
                                          binom_sampleB,
                                          estimator=np.mean,
                                          n_iter=1000)
    
    pvalor, _ = hyphotesis_testing.get_pvalue(test_value, muestras, alpha=min_pval)
    pvalores.append(pvalor)

    if pvalor <= min_pval:
      count += 1

  ratio = count / total_its
  n_its += 100
  print(ratio,n_its)
print(n_its-100)

0.38 110
0.01 210
0.0 310
0.01 410
0.03 510
0.14 610
0.85 710
0.95 810
710


# **Conclusiones**

*  Se necesitan 710 muestras para una potencia de 85%, usando un nivel de  significancia igual o menor a 0.05. 
*  En otras palabras estariamos 85% seguros de que existen diferencias entre los valores de conversión del Producto A y Producto B. 
*  Observando los resultados de la simulación, podemos decir que la potencia del test se incrementa a medida que se incrementa el tamaño de la muestra. Por lo tanto, observamos que una muestra grande hará más probable la detección de diferencias significativas cuando éstas realmente existan.